#### Import Libraries

In [1]:
import json
import requests
from datetime import datetime, timezone, timedelta
from decimal import Decimal
import pandas as pd
import numpy as np
import time

#### Date and Time

In [2]:
start = time.time()

now = datetime.now(timezone.utc) + timedelta(hours = 6)
ts = now.isoformat()

### JSON Part

#### AirKaz Parse

In [3]:
url_airkaz = 'https://airkaz.org'
rq = requests.get(url_airkaz)

psource = rq.text

psource = rq.text
sindex = psource.find('sensors_data')
psource = psource[sindex:]
findex = psource.find('<div class="container-fluid">')
psource = psource[:findex]
lstart = psource.find('[')+1
lfin = psource.find(']')
psource = psource[lstart:lfin].split("},")

alldata = []
json_code = json.loads(psource[0]+"}")

for e in list(range(len(psource))):
    try:
        json_code = json.loads(psource[e]+"}")
        alldata.append(json_code)
    except:
        pass

data = []

for e in alldata:
    if e['hour'] == 'now':
        data.append(e)

for air in data:
    air['ts'] = ts

datalen = list(range(len(data)))

for e in datalen:
    uid = data[e]['lat']+data[e]['lng']+data[e]['ts']+'|AirKaz'
    data[e]['pointId'] = uid
    data[e]['source'] = 'AirKaz.org'

cities = ['Алматы', 'Астана', 'Нур-Султан']
airkaz = []

for d in data:
    if d['city'] in cities and d['status'] == 'active' and d['pm25'] != None:
        airkaz.append(d)

#### KazGidroMet Parse

In [4]:
api_kgmt = 'e7c28f4e2e167304cee3418502d0e18c3a7772a61edc0b12afa71d2e4f2e3581'
url_kgmt = 'http://93.185.75.19:4003/simple/averages/last?key='

alldata = requests.get(url_kgmt+api_kgmt).json()

almaty = [27, 59, 97, 26, 28]
astana = [14, 12, 4, 8, 7]
kz = almaty+astana

kgmt = []

for d in alldata:
    if d['stationId'] in kz and d['code'] == 'PM2.5' or d['code'] == 'PM10' or d['code'] == 'NO2' or d['code'] == 'NO' or d['code'] == 'CO'or d['code'] == 'SO2'or d['code'] == 'H2S':
        kgmt.append(d)

for air in kgmt:
    air['ts'] = ts
    air['pointId'] = str(air['stationId'])+air['id']+air['ts']+'|KGMT'
    air['lat'] = 43.258597
    air['lng'] = 76.932439
    air['adr'] = 'TBA'
    air['source'] = 'КазГидроМет'
    if air['stationId'] in almaty:
        air['city'] = 'Алматы'
    elif air['stationId'] in astana:
        air['city'] = 'Астана'

#### AirVisual Parse

In [5]:
api_av = '07204a95-9cab-4030-9c92-585b48409ce2'
url_av = 'http://api.airvisual.com/v2/city?city=almaty&state=almaty-qalasy&country=kazakhstan&key='
data = requests.get(url_av+api_av).json()
pollist = data.get('data').get('history').get('pollution')

for e in pollist:
    e['city'] = 'Алматы'
    e['city_en'] = 'Almaty'

url2_av = 'http://api.airvisual.com/v2/city?city=nur-sultan&state=nur-sultan&country=kazakhstan&key='
data2 = requests.get(url2_av+api_av).json()
pollist2 = data2.get('data').get('history').get('pollution')

for e in pollist2:
    e['city'] = 'Астана'
    e['city_en'] = 'Astana'

airvisual = []
airvisual.append(pollist[0])
airvisual.append(pollist2[0])

for e in airvisual:
    e['pointId'] = e['ts']+e['city_en']+'|Airvisual'
    e['stationId'] = e['ts']+e['city_en']
    e['source'] = 'AirVisual.com'
    e['lat'] = 43.258597
    e['lng'] = 76.932439
    e['adr'] = 'TBA'
    e['p2'] = e['p2'].get('conc')
    e['ts_scrap'] = ts

#### US Embassy Parse

In [6]:
us_key = 'ce61d4dc68ce4de97bfe3616304d5044bc60dc2e'
url_almaty = 'https://api.waqi.info/feed/almaty/?token='
url_astana = 'https://api.waqi.info/feed/astana/?token='

data_almaty = requests.get(url_almaty+us_key).json()
data_almatycl = dict()
data_almatycl['time'] = data_almaty['data']['time']['s']
data_almatycl['aqi'] = data_almaty['data']['aqi']
data_almatycl['pm25'] = data_almaty['data']['iaqi']['pm25']['v']
data_almatycl['ts'] = ts
data_almatycl['pointId'] = ts+'_'+'Almaty'
data_almatycl['city'] = 'Алматы'
data_almatycl['city_en'] = 'Almaty'

data_astana = requests.get(url_astana+us_key).json()
data_astanacl = dict()
data_astanacl['time'] = data_astana['data']['time']['s']
data_astanacl['aqi'] = data_astana['data']['aqi']
data_astanacl['pm25'] = data_astana['data']['iaqi']['pm25']['v']
data_astanacl['ts'] = ts
data_astanacl['pointId'] = ts+'_'+'Astana'
data_astanacl['city'] = "Астана"
data_astanacl['city_en'] = "Astana"

us = []
us.append(data_almatycl)
us.append(data_astanacl)

for e in us:
    e['pointId'] = e['ts']+e['city_en']+'|US'
    e['stationId'] = e['ts']+e['city_en']
    e['source'] = 'US Embassy'
    e['lat'] = 43.258597
    e['lng'] = 76.932439
    e['adr'] = 'TBA'

### AirData DataFrame

In [7]:
cols = ['pointId', 'Дата и время', 'Источник', 'Дата и время датчика', 'Город', 'ID датчика', 'Адрес датчика', 'Lat', 'Lng', 'code','value']
airdata = pd.DataFrame(columns = cols)

#### AirKaz DF

In [8]:
airkazjsondf = pd.DataFrame(airkaz)
airkazdf = airkazjsondf[['pointId', 'ts', 'source', 'date', 'city', 'id', 'name', 'lat', 'lng', 'pm25', 'pm10']]
airkazdf = airkazdf.melt(id_vars = ['pointId', 'ts', 'source', 'date', 'city', 'id', 'name', 'lat', 'lng'], var_name = 'PM', value_name = 'value')
airkazdf = airkazdf.rename(columns={"PM": "code"})
airkazdf.columns = cols
airdata = airdata.append(airkazdf)

#### KGMT DF

In [9]:
kgmtjsondf = pd.DataFrame(kgmt)
kgmtjsondf['value'] = kgmtjsondf['value']*1000
kgmtdf = kgmtjsondf[['pointId', 'ts', 'source', 'date', 'city', 'stationId', 'adr', 'lat', 'lng', 'code','value']]
kgmtdf.columns = cols
airdata = airdata.append(kgmtdf)

#### AirVisual DF

In [10]:
avjsondf = pd.DataFrame(airvisual)
avdf = avjsondf[['pointId', 'ts_scrap', 'source', 'ts', 'city', 'stationId', 'adr', 'lat', 'lng','mainus', 'p2']]
avdf.columns = cols
airdata = airdata.append(avdf)

#### US Embassy DF

In [11]:
usjsondf = pd.DataFrame(us)
usjsondf['code'] = 'pm25'
usdf = usjsondf[['pointId', 'ts', 'source', 'time', 'city', 'stationId', 'adr', 'lat', 'lng', 'code', 'pm25']]
usdf.columns = cols
airdata = airdata.append(usdf)

In [12]:
airdata['value'] = airdata['value'].astype(float)
airdata['Lat'] = airdata['Lat'].astype(float)
airdata['Lng'] = airdata['Lng'].astype(float)

In [13]:
end = time.time()
print(end-start)

7.847209930419922


In [14]:
airdata['Город'] = airdata['Город'].replace('Астана', 'Нур-Султан')
airdata['code'] = airdata['code'].replace('pm10', 'PM10')
airdata['code'] = airdata['code'].replace('pm25', 'PM2.5')

In [15]:
airdata.head()

,pointId,Дата и время,Источник,Дата и время датчика,Город,ID датчика,Адрес датчика,Lat,Lng,code,value
0,43.253076.91002020-05-03T20:41:49.109095+00:00...,2020-05-03T20:41:49.109095+00:00,AirKaz.org,2020-05-03 20:39:07,Алматы,5,ТолеБи - Байзакова,43.253,76.910,PM2.5,10.27
1,43.214076.89302020-05-03T20:41:49.109095+00:00...,2020-05-03T20:41:49.109095+00:00,AirKaz.org,2020-05-03 20:39:11,Алматы,6,Розыбакиева - Байкадамова,43.214,76.893,PM2.5,1.46
2,43.296076.84402020-05-03T20:41:49.109095+00:00...,2020-05-03T20:41:49.109095+00:00,AirKaz.org,2020-05-03 20:39:13,Алматы,7,Кок Кайнар,43.296,76.844,PM2.5,83.00
3,43.174076.91702020-05-03T20:41:49.109095+00:00...,2020-05-03T20:41:49.109095+00:00,AirKaz.org,2020-05-03 20:39:07,Алматы,9,Ерменсай,43.174,76.917,PM2.5,1.55
4,43.177076.96602020-05-03T20:41:49.109095+00:00...,2020-05-03T20:41:49.109095+00:00,AirKaz.org,2020-05-03 20:39:07,Алматы,12,Каменское плато,43.177,76.966,PM2.5,4.67
